# Model Distributions

Using the static agent model, comparing the distributions derived from different decisionmaking models.

In [1]:
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from small_crisis import CrisisModel, CrisisAgent, CrisisWorld

from consul.agent_types.rl_agent import RLAgent
from consul.agent_types.cbr_agent import CaseBased_RLAgent


## Setting up agent and model types

In [2]:
class LearningAgent(RLAgent, CrisisAgent):
    pass

In [3]:
class CBRLPlayer(CaseBased_RLAgent, CrisisAgent):
    pass

In [4]:
class CBCrisisModel(CrisisModel):
    def __init__(self, agents):
        agent1, agent2 = agents
        if hasattr(agent1, "set_new_case"):
            agent1.set_new_case(self.make_coordinates(agent2))
            agent2.set_new_case(self.make_coordinates(agent1))
        super().__init__(agents)
    
    @staticmethod
    def make_coordinates(agent):
        '''
        Extract a coordinate tuple from a crisis agent
        '''
        return (agent.mil_strength, agent.assets, agent.bloc * 100)

## Experiment 1.1 -- 10 agents, $\alpha=0.1$

In [6]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 10

## Experiment 1.2 -- 20 agents, $\alpha=0.1$

In [12]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 20

In [13]:
rl_models = []
cbrl_models = []

for i in range(100):
    seed = random.randint(1000, 100000)
    model_1 = CrisisWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
    model_2 = CrisisWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
    model_2.model_class = CBCrisisModel
    
    model_1.run(100)
    model_2.run(100)
    rl_models.append(model_1)
    cbrl_models.append(model_2)
    
    if i % 10 == 0:
        print(i)

0
10


KeyboardInterrupt: 

In [ ]:
rl_tables = []
cbrl_tables = []

for i in range(100):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)

In [ ]:
rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)

In [ ]:
with open("data_out/ex_1.1.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table), f)

## Experiment 1.3 - Vary learning rates

In [14]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 10
num_iter = 100

In [15]:
rl_models = []
cbrl_models = []

for learning_rate in np.linspace(0.1, 1, 10):
    agent_args = {"learning_rate": learning_rate,
              "discount_factor": 1-learning_rate}
    
    for i in range(num_iter):
        seed = random.randint(1000, 100000)
        model_1 = CrisisWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
        model_2 = CrisisWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
        model_2.model_class = CBCrisisModel

        model_1.run(100)
        model_2.run(100)
        rl_models.append(model_1)
        cbrl_models.append(model_2)
    
    print(learning_rate)
    

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [16]:
rl_tables = []
cbrl_tables = []
run_table = []

for i in range(len(rl_models)):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)
    
    run_row = m1.agent_args.copy()
    run_row["Seed"] = m1.seed
    run_table.append(run_row)

In [17]:
rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)
seed_table = pd.DataFrame(run_table)
with open("data_out/ex_1.3.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table, seed_table), f)

In [68]:
# Sidebar: how likely was I to get a seed collission?
p_nocol = 1
for i in range(499):
    p_nocol *= (999000-i)/999000
print(1 - p_nocol)


0.11696996856182307
